In [1]:
from collections import deque

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import nltk
from torch.utils.data import Dataset
import pickle

from utils.utils import *
from utils.label_decoding import *
from utils.HierarchicalLoss import *

# SubTask 1

In [2]:
class DataSet(Dataset):
    def __init__(self, df, labels_at_level, features_file):
        super(DataSet, self).__init__()
        self.data_df = df
        self.labels_at_level = labels_at_level
        self.features_file = features_file
        self.features_dict = None
        with open(features_file, 'rb') as f:
            self.features_dict = pickle.load(f)
    
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        id = self.data_df.iloc[idx]['id']
        text = self.data_df.iloc[idx]['cleaned_text']
        level_1_target = self.encode(self.data_df.iloc[idx]['Level 1'], 1)
        level_2_target = self.encode(self.data_df.iloc[idx]['Level 2'], 2)
        level_3_target = self.encode(self.data_df.iloc[idx]['Level 3'], 3)
        level_4_target = self.encode(self.data_df.iloc[idx]['Level 4'], 4)
        level_5_target = self.encode(self.data_df.iloc[idx]['Level 5'], 5)
            
        
        return {'id': id,
                'text': text, 
                'text_features': self.features_dict[id],
                'level_1_target': level_1_target, 
                'level_2_target': level_2_target, 
                'level_3_target': level_3_target, 
                'level_4_target': level_4_target, 
                'level_5_target': level_5_target }

    def encode(self, labels, level):
        level_ = f'Level {level}'
        
        target = torch.zeros(len(self.labels_at_level[level_])+1)
        
        for label in labels:
            label_idx = self.labels_at_level[level_][label]
            target[label_idx] = 1
        
        if len(labels) == 0:
            target[-1] = 1
        
        return target

In [3]:
class TestDataSet(Dataset):
    def __init__(self, df, features_file):
        super(TestDataSet, self).__init__()
        self.data_df = df
        self.features_file = features_file
        self.features_dict = None
        with open(features_file, 'rb') as f:
            self.features_dict = pickle.load(f)
    
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        id = self.data_df.iloc[idx]['id']
        text = self.data_df.iloc[idx]['cleaned_text']
        
        return {'id': id,
                'text': text, 
                'text_features': self.features_dict[id] }

In [4]:
class Baseline(torch.nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        
        self.linear_level1 = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU(), nn.Linear(512, 128), nn.ReLU())
        self.linear_level2 = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU(), nn.Linear(512, 128), nn.ReLU())
        self.linear_level3 = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU(), nn.Linear(512, 128), nn.ReLU())
        self.linear_level4 = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU(), nn.Linear(512, 128), nn.ReLU())
        self.linear_level5 = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU(), nn.Linear(512, 128), nn.ReLU())

        self.sigmoid_reg1 = nn.Sequential(nn.Linear(128, 2))
        self.sigmoid_reg2 = nn.Sequential(nn.Linear(128*2, 4))
        self.sigmoid_reg3 = nn.Sequential(nn.Linear(128*3, 13), nn.Sigmoid())
        self.sigmoid_reg4 = nn.Sequential(nn.Linear(128*4, 13), nn.Sigmoid())
        self.sigmoid_reg5 = nn.Sequential(nn.Linear(128*5, 7), nn.Sigmoid())
        
        
    def forward(self, embeddings):
        
        lvl1_rep = self.linear_level1(embeddings)
        lvl2_rep = self.linear_level2(embeddings)
        lvl3_rep = self.linear_level3(embeddings)
        lvl4_rep = self.linear_level4(embeddings)
        lvl5_rep = self.linear_level5(embeddings)
        
        lvl1_pred = self.sigmoid_reg1(lvl1_rep)
        lvl2_pred = self.sigmoid_reg2(torch.cat((lvl1_rep, lvl2_rep), dim=1))
        lvl3_pred = self.sigmoid_reg3(torch.cat((lvl1_rep, lvl2_rep, lvl3_rep), dim=1))
        lvl4_pred = self.sigmoid_reg4(torch.cat((lvl1_rep, lvl2_rep, lvl3_rep, lvl4_rep), dim=1))
        lvl5_pred = self.sigmoid_reg5(torch.cat((lvl1_rep, lvl2_rep, lvl3_rep, lvl4_rep, lvl5_rep), dim=1))
        
        return lvl1_pred, lvl2_pred, lvl3_pred, lvl4_pred, lvl5_pred

In [5]:
class Baseline(torch.nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        
        self.linear_level1 = nn.Sequential(nn.Linear(1024, 2048), nn.ReLU(), nn.Linear(2048, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU())
        self.linear_level2 = nn.Sequential(nn.Linear(1024, 2048), nn.ReLU(), nn.Linear(2048, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU())
        self.linear_level3 = nn.Sequential(nn.Linear(1024, 2048), nn.ReLU(), nn.Linear(2048, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU())
        self.linear_level4 = nn.Sequential(nn.Linear(1024, 2048), nn.ReLU(), nn.Linear(2048, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU())
        self.linear_level5 = nn.Sequential(nn.Linear(1024, 2048), nn.ReLU(), nn.Linear(2048, 1024), nn.ReLU(), nn.Linear(1024, 512), nn.ReLU())

        self.sigmoid_reg1 = nn.Sequential(nn.Linear(512, 2))
        self.sigmoid_reg2 = nn.Sequential(nn.Linear(512*2, 4))
        self.sigmoid_reg3 = nn.Sequential(nn.Linear(512*3, 13), nn.Sigmoid())
        self.sigmoid_reg4 = nn.Sequential(nn.Linear(512*4, 13), nn.Sigmoid())
        self.sigmoid_reg5 = nn.Sequential(nn.Linear(512*5, 7), nn.Sigmoid())
        
        
    def forward(self, embeddings):
        
        lvl1_rep = self.linear_level1(embeddings)
        lvl2_rep = self.linear_level2(embeddings)
        lvl3_rep = self.linear_level3(embeddings)
        lvl4_rep = self.linear_level4(embeddings)
        lvl5_rep = self.linear_level5(embeddings)
        
        lvl1_pred = self.sigmoid_reg1(lvl1_rep)
        lvl2_pred = self.sigmoid_reg2(torch.cat((lvl1_rep, lvl2_rep), dim=1))
        lvl3_pred = self.sigmoid_reg3(torch.cat((lvl1_rep, lvl2_rep, lvl3_rep), dim=1))
        lvl4_pred = self.sigmoid_reg4(torch.cat((lvl1_rep, lvl2_rep, lvl3_rep, lvl4_rep), dim=1))
        lvl5_pred = self.sigmoid_reg5(torch.cat((lvl1_rep, lvl2_rep, lvl3_rep, lvl4_rep, lvl5_rep), dim=1))
        
        return lvl1_pred, lvl2_pred, lvl3_pred, lvl4_pred, lvl5_pred

In [6]:
def evaluate_model(model, dataloader, pred_file_path, gold_file_path, 
                   evaluator_script_path, id2leaf_label, format=None,validation=False):
    model.eval()
    predictions = []
    
    HL = HierarchicalLoss(id2label=id2label_1, hierarchical_labels=hierarchy_1, persuasion_techniques=persuasion_techniques_1, device=device)
    total_loss = 0
    
    
    with torch.no_grad():
        
        for batch in dataloader:
            if not isinstance(batch['id'], list):
                ids = batch['id'].detach().numpy().tolist()
            else:
                ids = batch['id']
        
            embeddings = batch['text_features']
            embeddings = embeddings.to(device)
            pred_1, pred_2, pred_3, pred_4, pred_5 = model(embeddings)
            
            if validation:
                y_1, y_2, y_3 = batch['level_1_target'], batch['level_2_target'], batch['level_3_target']
                y_4, y_5 = batch['level_4_target'], batch['level_5_target']
                
                y_1, y_2, y_3, y_4, y_5 = y_1.to(device), y_2.to(device), y_3.to(device), y_4.to(device), y_5.to(device)
                
                dloss = HL.calculate_dloss([pred_1, pred_2, pred_3, pred_4, pred_5], [y_1, y_2, y_3, y_4, y_5])
                lloss = HL.calculate_lloss([pred_1, pred_2, pred_3, pred_4, pred_5], [y_1, y_2, y_3, y_4, y_5])
                
                total_loss += (dloss + lloss).detach().cpu().item()
                
            pred_3 = (pred_3.cpu().detach().numpy() > 0.5).astype(int)
            pred_4 = (pred_4.cpu().detach().numpy() > 0.5).astype(int)
            pred_5 = (pred_5.cpu().detach().numpy() > 0.5).astype(int)
            
            predictions += get_labels(id2leaf_label, ids, pred_3, pred_4, pred_5, format)

        # Writing JSON data
        with open(pred_file_path, 'w') as f:
            json.dump(predictions, f, indent=4)
        
        if gold_file_path is None:
            return
        
        prec_h, rec_h, f1_h = evaluate_h(pred_file_path, gold_file_path)
        # print("f1_h={:.5f}\tprec_h={:.5f}\trec_h={:.5f}".format(f1_h, prec_h, rec_h))
        if validation:
            return prec_h, rec_h, f1_h, total_loss / len(dataloader)
            
        # command = [
        #         "python3", evaluator_script_path,
        #         "--gold_file_path", gold_file_path,
        #         "--pred_file_path", pred_file_path
        # ]
        # 
        # result = subprocess.run(command, capture_output=True, text=True)
        # 
        # if result.returncode == 0:
        #     print("Output:\n", result.stdout)
        # else:
        #     print("Error:\n", result.stderr)
        #     
        
    

In [7]:
from torch.utils.data import DataLoader

train_json = './semeval2024_dev_release/subtask1/train.json'
validation_json = './semeval2024_dev_release/subtask1/validation.json'

train_data = process_json(train_json, techniques_to_level_1, hierarchy_1)
# val_data = 
validation_data = process_json(validation_json, techniques_to_level_1, hierarchy_1)


training_dataset = DataSet(train_data, indexed_persuasion_techniques_1, 
                           './TextFeatures/subtask1a/XLM-RoBERTa/train_text_features.pkl')
validation_dataset = DataSet(validation_data, indexed_persuasion_techniques_1, 
                             './TextFeatures/subtask1a/XLM-RoBERTa/validation_text_features.pkl')

In [8]:
device = get_device()

device = torch.device('cpu')

HL = HierarchicalLoss(id2label=id2label_1, hierarchical_labels=hierarchy_1, persuasion_techniques=persuasion_techniques_1, device=device)

Using MPS


In [9]:
import wandb

# Initialize WandB and log in to your account
wandb.login()

sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-5,
            'max': 1e-3
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
        'optimizer': {
            'values': ['adam']
        },
        'beta1': {  # Relevant for Adam
            'min': 0.85,
            'max': 0.95
        },
        'momentum': {  # Relevant for SGD
            'min': 0.8,
            'max': 0.99
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="FineTuning-XLM-RoBERTa")
# sweep_id = '44uz6ydx'

wandb: Currently logged in as: iqbal_shaik (phoenix_nlp). Use `wandb login --relogin` to force relogin


Create sweep with ID: berwitur
Sweep URL: https://wandb.ai/phoenix_nlp/FineTuning-XLM-RoBERTa/sweeps/berwitur


In [10]:
models_dir = './models/subtask1a/XLM-RoBERTa-nn/'
num_epochs = 300

In [11]:
from tqdm import tqdm
import json
import subprocess
from subtask_1_2a import *

def train():
    
    wandb.init()

    # Use WandB configurations
    config = wandb.config
    batch_size = config.batch_size
    learning_rate = config.learning_rate
    
    train_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
    
    model = Baseline()
    model.to(device)
    
    optimizer = None
    
    if config.optimizer == 'adam':
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=learning_rate,
            betas=(config.beta1, 0.999)
        )
    elif config.optimizer == 'sgd':
        optimizer = torch.optim.SGD(
            model.parameters(),
            lr=learning_rate,
            momentum=config.momentum
        )
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for batch_idx, batch in enumerate(train_dataloader):
            
            y_1, y_2, y_3 = batch['level_1_target'], batch['level_2_target'], batch['level_3_target']
            y_4, y_5 = batch['level_4_target'], batch['level_5_target']
            
            embeddings = batch['text_features']
            embeddings = embeddings.to(device)
            y_1, y_2, y_3, y_4, y_5 = y_1.to(device), y_2.to(device), y_3.to(device), y_4.to(device), y_5.to(device)
            
            
            optimizer.zero_grad()
            pred_1, pred_2, pred_3, pred_4, pred_5 = model(embeddings)
            
            dloss = HL.calculate_dloss([pred_1, pred_2, pred_3, pred_4, pred_5], [y_1, y_2, y_3, y_4, y_5])
            lloss = HL.calculate_lloss([pred_1, pred_2, pred_3, pred_4, pred_5], [y_1, y_2, y_3, y_4, y_5])
    
            total_loss = lloss + dloss
            
            total_loss.backward()
            optimizer.step()
            
            running_loss += total_loss.detach().item()
            
            # if batch_idx % 20 == 19:
            #     print(f"[{epoch + 1}, {batch_idx + 1}] loss: {running_loss / 20:.3f}")
            #     running_loss = 0.0
        
        running_loss /= len(train_dataloader)
        
        val_pred_file_path = './Predictions/val_predictions_subtask1.json'
        val_gold_file_path = './semeval2024_dev_release/subtask1/validation.json'
        evaluator_script = './scorer-baseline/subtask_1_2a.py'
        
        prec_h, rec_h, f1_h, validation_loss = evaluate_model(model, validation_dataloader, val_pred_file_path, 
                                         val_gold_file_path, evaluator_script,id2leaf_label,
                                         validation=True)
        
        if epoch % 100 == 99:
            print(f'[{epoch+1}/{num_epochs}]')
            print("f1_h={:.5f}\tprec_h={:.5f}\trec_h={:.5f}".format(f1_h, prec_h, rec_h))
            
        
        # Log training metrics
        wandb.log({"epoch": epoch, "train_loss": running_loss})
        wandb.log({"val_loss": validation_loss})
        wandb.log({"h_precision": prec_h, "h_recall": rec_h, "h_f1-score":f1_h})
        
    
    torch.save(model.state_dict(), f"{models_dir}{wandb.run.name}.pth")
    wandb.join()

In [12]:
def main():
    wandb.agent(sweep_id, train, count=10)

if __name__ == "__main__":
    main()

wandb: Agent Starting Run: 30am8a20 with config:
wandb: 	batch_size: 64
wandb: 	beta1: 0.8945734326666321
wandb: 	learning_rate: 0.00014414356730831033
wandb: 	momentum: 0.9246751923533026
wandb: 	optimizer: adam


[100/300]
f1_h=0.44803	prec_h=0.56928	rec_h=0.36935
[200/300]
f1_h=0.46916	prec_h=0.53232	rec_h=0.41940
[300/300]
f1_h=0.50880	prec_h=0.53353	rec_h=0.48627


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
h_f1-score,▁▂▃▅▆▆▆▇▇▇▇▆▇▇█▇▇█▇███▇▇▇▇▇██▇██████████
h_precision,▂█▆▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▂▁▁▁▁▁▂▁▁▂▁▁
h_recall,▁▂▂▃▅▅▅▆▅▅▆▅▇▆▇▇▆▇▆▇▇█▇▇▇▇▇▇▇▆██▇▇▇█▇▇██
train_loss,██▇▆▆▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇█▇▅▆▂▁▂▂▃▂▆▄▃▂▁▃▂▂▂▃▂▅▄▃▂▄▁▂▁▃▄▄▃▄▄▆▂▃▅
epoch,299
h_f1-score,0.5088
h_precision,0.53353
h_recall,0.48627
train_loss,294.07921


wandb: Agent Starting Run: 1iuu3azu with config:
wandb: 	batch_size: 256
wandb: 	beta1: 0.9405925541813452
wandb: 	learning_rate: 0.0006797736286448581
wandb: 	momentum: 0.9376595760094972
wandb: 	optimizer: adam


[100/300]
f1_h=0.44807	prec_h=0.58028	rec_h=0.36492
[200/300]
f1_h=0.48991	prec_h=0.53515	rec_h=0.45173
[300/300]
f1_h=0.45941	prec_h=0.51866	rec_h=0.41231


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
h_f1-score,▁▃▆▆▅▆▇▆▇█▇████▇███████▇██████▇█▇██▇█▇██
h_precision,█▆▄▄▄▄▃▃▂▂▂▃▂▂▂▂▁▂▁▁▂▁▁▁▁▂▁▁▂▁▂▁▁▁▂▂▂▁▁▂
h_recall,▁▂▄▅▄▅▆▅▆▇▆▇▇▇▇▇█▇█▇▇▇█▇▇▇▇█▇█▇▇▇█▇▆▇▇█▇
train_loss,█▇▇▆▆▆▅▅▄▄▄▄▃▃▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▅▄▁▃▃▁▁▄▂▃▃▃▃▃▅▄▅▅▄▄▄▄▆█▄▅▄▆▄▆▅▇▄▄▆▅▆▆▄
epoch,299
h_f1-score,0.47557
h_precision,0.5514
h_recall,0.41807
train_loss,287.87083


wandb: Agent Starting Run: t2m890j4 with config:
wandb: 	batch_size: 64
wandb: 	beta1: 0.8683018926344438
wandb: 	learning_rate: 5.165546626884552e-05
wandb: 	momentum: 0.9701328215709653
wandb: 	optimizer: adam


wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.


[100/300]
f1_h=0.41006	prec_h=0.70716	rec_h=0.28875
[200/300]
f1_h=0.46918	prec_h=0.58632	rec_h=0.39105


wandb: Ctrl + C detected. Stopping sweep.
